# Pandasaurus CxG Extension Walkthrough 

## Overview
Welcome to this Jupyter notebook walkthrough for pandasaurus_cxg! This library provides powerful tools for analyzing and enriching AnnData objects, enabling you to gain deeper insights into your single-cell RNA sequencing (scRNA-seq) data.

In this notebook, we will explore two main classes: `AnndataEnricher` and `AnndataAnalyzer`. Let's dive in and see how these classes can help us in our scRNA-seq analysis.

Now, let's get started with an example workflow that demonstrates the capabilities of these classes. We'll load an example dataset, perform enrichment, analysis, and visualization steps to gain a better understanding of our scRNA-seq data.

## Test Data
The following files are used in the walkthrough. Please download them manually to a folder of your choice. Ensure that you adjust the file paths used in the examples to match your local file paths.
- [Time-resolved Systems Immunology Reveals a Late Juncture Linked to Fatal COVID-19: Adaptive Cells](https://cellxgene.cziscience.com/collections/db14ce52-5dd6-4649-a9e9-7fb2572d0605)
- [Integrated Single-nucleus and Single-cell RNA-seq of the Adult Human Kidney](https://cellxgene.cziscience.com/collections/36b8480d-114e-42fe-b6a9-bdf79a7eb1fc)

## AnndataEnricher Walkthrough

### Initialization
Let's import the necessary modules and initialize our AnndataEnricher

In [1]:
from pandasaurus_cxg.anndata_enricher import AnndataEnricher

In [2]:
# Using Time-resolved Systems Immunology Reveals a Late Juncture Linked to Fatal COVID-19: Adaptive Cells dataset
ade = AnndataEnricher.from_file_path("test/data/test_covid.h5ad")

Anndata Obs field details

In [3]:
ade._anndata.obs

,tissue_ontology_term_id,author_cell_type,disease_ontology_term_id,age,days_since_hospitalized,donor_id,severity,dsm_severity_score,ever_admitted_to_icu,days_since_onset,...,organism_ontology_term_id,suspension_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage
AAACCTGAGACGACGT-1_1,UBERON:0000178,naive B cell,PATO:0000461,32.0,,AA220907,,NaN,,,...,NCBITaxon:9606,cell,naive B cell,10x 5' v1,normal,Homo sapiens,female,blood,unknown,32-year-old human stage
AAACCTGAGCTAGTTC-1_1,UBERON:0000178,naive B cell,MONDO:0100096,37.0,6.0,HGR0000079,Critical,NaN,False,30.0,...,NCBITaxon:9606,cell,naive B cell,10x 5' v1,COVID-19,Homo sapiens,female,blood,European,37-year-old human stage
AAACCTGCATAGACTC-1_1,UBERON:0000178,memory B cell,PATO:0000461,32.0,,AA220907,,NaN,,,...,NCBITaxon:9606,cell,memory B cell,10x 5' v1,normal,Homo sapiens,female,blood,unknown,32-year-old human stage
AAACCTGCATTAACCG-1_1,UBERON:0000178,naive B cell,MONDO:0100096,54.0,1.0,HGR0000143,Critical,1.674056,True,8.0,...,NCBITaxon:9606,cell,naive B cell,10x 5' v1,COVID-19,Homo sapiens,female,blood,East Asian,54-year-old human stage
AAACCTGGTCATCGGC-1_1,UBERON:0000178,gamma-delta T cell,MONDO:0100096,55.0,1.0,HGR0000083,Moderate,-1.950858,False,6.0,...,NCBITaxon:9606,cell,gamma-delta T cell,10x 5' v1,COVID-19,Homo sapiens,male,blood,European,55-year-old human stage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCACAAGAGGCT-16_4,UBERON:0000178,memory B cell,PATO:0000461,73.0,,SHD5,,NaN,,,...,NCBITaxon:9606,cell,memory B cell,10x 5' v1,normal,Homo sapiens,male,blood,unknown,73-year-old human stage
TTTGTCACACGTCTCT-16_4,UBERON:0000178,naive CD8+ T cell,MONDO:0100096,49.0,0.0,HGR0000142,Critical,-0.617735,False,12.0,...,NCBITaxon:9606,cell,"naive thymus-derived CD8-positive, alpha-beta ...",10x 5' v1,COVID-19,Homo sapiens,male,blood,European,49-year-old human stage
TTTGTCACAGGTCGTC-16_4,UBERON:0000178,naive CD8+ T cell,MONDO:0100096,53.0,2.0,HGR0000134,Critical,-1.925448,False,16.0,...,NCBITaxon:9606,cell,"naive thymus-derived CD8-positive, alpha-beta ...",10x 5' v1,COVID-19,Homo sapiens,male,blood,European,53-year-old human stage
TTTGTCAGTAAATGAC-16_4,UBERON:0000178,"CD4-positive, alpha-beta memory T cell",PATO:0000461,51.0,,SHD1,,NaN,,,...,NCBITaxon:9606,cell,"CD4-positive, alpha-beta memory T cell",10x 5' v1,normal,Homo sapiens,male,blood,unknown,51-year-old human stage


Avialable slims for minimal and full slim enrichment methods

In [4]:
ade.slim_list

[{'name': 'blood_and_immune_upper_slim',
  'description': 'a subset of general classes related to blood and the immune system, primarily of hematopoietic origin'},
 {'name': 'eye_upper_slim',
  'description': 'a subset of general classes related to specific cell types in the eye.'},
 {'name': 'general_cell_types_upper_slim',
  'description': 'a subset of general classes of cell types in the cell ontology.'}]

Avialable slims for contextual enrichment methods

In [5]:
ade._AnndataEnricher__context_list

['UBERON:0000178']

### Enrichment

Show all cells that are instances of "memory T cell" - before enrichment

In [6]:
ade._anndata.obs[ade._anndata.obs["cell_type_ontology_term_id"] == "CL:0000813"]

,tissue_ontology_term_id,author_cell_type,disease_ontology_term_id,age,days_since_hospitalized,donor_id,severity,dsm_severity_score,ever_admitted_to_icu,days_since_onset,...,organism_ontology_term_id,suspension_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage
AGTCTTTTCTGGTGTA-1_1,UBERON:0000178,TissueResMemT,PATO:0000461,47.0,,AA220534,,NaN,,,...,NCBITaxon:9606,cell,memory T cell,10x 5' v1,normal,Homo sapiens,male,blood,unknown,47-year-old human stage
CGGGTCATCAGGCCCA-1_1,UBERON:0000178,TissueResMemT,PATO:0000461,47.0,,AA220534,,NaN,,,...,NCBITaxon:9606,cell,memory T cell,10x 5' v1,normal,Homo sapiens,male,blood,unknown,47-year-old human stage
CTACATTTCACATACG-1_1,UBERON:0000178,TissueResMemT,MONDO:0100096,52.0,9.0,HGR0000074,Critical,-0.626780,False,18.0,...,NCBITaxon:9606,cell,memory T cell,10x 5' v1,COVID-19,Homo sapiens,male,blood,European,52-year-old human stage
GTGCAGCAGCGCCTCA-1_1,UBERON:0000178,TissueResMemT,PATO:0000461,32.0,,AA220907,,NaN,,,...,NCBITaxon:9606,cell,memory T cell,10x 5' v1,normal,Homo sapiens,female,blood,unknown,32-year-old human stage
TGGCTGGAGTAGTGCG-1_1,UBERON:0000178,TissueResMemT,PATO:0000461,47.0,,AA220534,,NaN,,,...,NCBITaxon:9606,cell,memory T cell,10x 5' v1,normal,Homo sapiens,male,blood,unknown,47-year-old human stage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCCCGATAGGTGATTA-16_4,UBERON:0000178,TissueResMemT,MONDO:0100096,54.0,12.0,HGR0000429,Severe,-1.189216,False,28.0,...,NCBITaxon:9606,cell,memory T cell,10x 5' v1,COVID-19,Homo sapiens,male,blood,unknown,54-year-old human stage
TCTGGAAAGAGTAATC-16_4,UBERON:0000178,TissueResMemT,MONDO:0100096,55.0,13.0,HGR0000051,Critical,0.082405,False,20.0,...,NCBITaxon:9606,cell,memory T cell,10x 5' v1,COVID-19,Homo sapiens,male,blood,European,55-year-old human stage
TGACTTTTCCACTCCA-16_4,UBERON:0000178,TissueResMemT,PATO:0000461,58.0,,SHD3,,NaN,,,...,NCBITaxon:9606,cell,memory T cell,10x 5' v1,normal,Homo sapiens,male,blood,unknown,58-year-old human stage
TGCACCTTCTAACTTC-16_4,UBERON:0000178,TissueResMemT,PATO:0000461,58.0,,SHD3,,NaN,,,...,NCBITaxon:9606,cell,memory T cell,10x 5' v1,normal,Homo sapiens,male,blood,unknown,58-year-old human stage


#### Simple enrichment
Returns a DataFrame that is enriched with inferred relationships between terms in the seed. Subject and object terms are members of the seed terms.

In [7]:
simple_enrichment = ade.simple_enrichment()
simple_enrichment

,s,s_label,p,o,o_label
0,CL:0000798,gamma-delta T cell,rdfs:subClassOf,CL:0000084,T cell
1,CL:0000809,"double-positive, alpha-beta thymocyte",rdfs:subClassOf,CL:0000084,T cell
2,CL:0000813,memory T cell,rdfs:subClassOf,CL:0000084,T cell
3,CL:0000815,regulatory T cell,rdfs:subClassOf,CL:0000084,T cell
4,CL:0000895,"naive thymus-derived CD4-positive, alpha-beta ...",rdfs:subClassOf,CL:0000084,T cell
5,CL:0000897,"CD4-positive, alpha-beta memory T cell",rdfs:subClassOf,CL:0000084,T cell
6,CL:0000897,"CD4-positive, alpha-beta memory T cell",rdfs:subClassOf,CL:0000813,memory T cell
7,CL:0000900,"naive thymus-derived CD8-positive, alpha-beta ...",rdfs:subClassOf,CL:0000084,T cell
8,CL:0000909,"CD8-positive, alpha-beta memory T cell",rdfs:subClassOf,CL:0000084,T cell
9,CL:0000909,"CD8-positive, alpha-beta memory T cell",rdfs:subClassOf,CL:0000813,memory T cell


Show all cells that are instances of "memory T cell" - after enrichment

In [8]:
t_cell_instances = simple_enrichment[simple_enrichment["o_label"] == "memory T cell"]["s"].tolist()
t_cell_instances.append("CL:0000813")
ade._anndata.obs[ade._anndata.obs["cell_type_ontology_term_id"].isin(t_cell_instances)]

,tissue_ontology_term_id,author_cell_type,disease_ontology_term_id,age,days_since_hospitalized,donor_id,severity,dsm_severity_score,ever_admitted_to_icu,days_since_onset,...,organism_ontology_term_id,suspension_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage
AAAGCAAGTGAGTGAC-1_1,UBERON:0000178,"CD4-positive, alpha-beta memory T cell",MONDO:0100096,57.0,1.0,HGR0000136,Moderate,-2.910737,False,17.0,...,NCBITaxon:9606,cell,"CD4-positive, alpha-beta memory T cell",10x 5' v1,COVID-19,Homo sapiens,female,blood,European,57-year-old human stage
AAAGTAGTCCTATTCA-1_1,UBERON:0000178,"CD4-positive, alpha-beta memory T cell",MONDO:0100096,35.0,3.0,HGR0000091,Moderate,-1.418131,False,20.0,...,NCBITaxon:9606,cell,"CD4-positive, alpha-beta memory T cell",10x 5' v1,COVID-19,Homo sapiens,male,blood,European,35-year-old human stage
AACACGTGTGTATGGG-1_1,UBERON:0000178,"CD8-positive, alpha-beta memory T cell",MONDO:0100096,40.0,18.0,HGR0000078,Critical,-0.092375,True,23.0,...,NCBITaxon:9606,cell,"CD8-positive, alpha-beta memory T cell",10x 5' v1,COVID-19,Homo sapiens,female,blood,European,40-year-old human stage
AACCATGAGCGTGAAC-1_1,UBERON:0000178,"CD8-positive, alpha-beta memory T cell",PATO:0000461,47.0,,AA220534,,NaN,,,...,NCBITaxon:9606,cell,"CD8-positive, alpha-beta memory T cell",10x 5' v1,normal,Homo sapiens,male,blood,unknown,47-year-old human stage
AACCATGCAGCCACCA-1_1,UBERON:0000178,"CD4-positive, alpha-beta memory T cell",MONDO:0100096,79.0,1.0,HGR0000137,Severe,-1.201561,False,4.0,...,NCBITaxon:9606,cell,"CD4-positive, alpha-beta memory T cell",10x 5' v1,COVID-19,Homo sapiens,female,blood,European,79-year-old human stage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTAGTTCGCAT-16_4,UBERON:0000178,"CD4-positive, alpha-beta memory T cell",MONDO:0100096,54.0,10.0,HGR0000429,Severe,-1.189216,False,26.0,...,NCBITaxon:9606,cell,"CD4-positive, alpha-beta memory T cell",10x 5' v1,COVID-19,Homo sapiens,male,blood,unknown,54-year-old human stage
TTTGGTTCATGCCACG-16_4,UBERON:0000178,TissueResMemT,MONDO:0100096,54.0,12.0,HGR0000429,Severe,-1.189216,False,28.0,...,NCBITaxon:9606,cell,memory T cell,10x 5' v1,COVID-19,Homo sapiens,male,blood,unknown,54-year-old human stage
TTTGGTTCATTACCTT-16_4,UBERON:0000178,"CD4-positive, alpha-beta memory T cell",PATO:0000461,51.0,,SHD1,,NaN,,,...,NCBITaxon:9606,cell,"CD4-positive, alpha-beta memory T cell",10x 5' v1,normal,Homo sapiens,male,blood,unknown,51-year-old human stage
TTTGTCAAGTGGACGT-16_4,UBERON:0000178,"CD4-positive, alpha-beta memory T cell",MONDO:0100096,55.0,10.0,HGR0000051,Critical,0.082405,False,17.0,...,NCBITaxon:9606,cell,"CD4-positive, alpha-beta memory T cell",10x 5' v1,COVID-19,Homo sapiens,male,blood,European,55-year-old human stage


#### Minimal slim enrichment
Returns a DataFrame that is enriched with inferred relationships between terms in the seed list and in an extended seed list. The extended seed list consists of terms from the seed list and terms from given slim lists, classes tagged with some specified ‘subset’ axiom.

In [9]:
ade.minimal_slim_enrichment(["blood_and_immune_upper_slim"])

,s,s_label,p,o,o_label
0,CL:0000084,T cell,rdfs:subClassOf,CL:0000842,mononuclear cell
1,CL:0000787,memory B cell,rdfs:subClassOf,CL:0000842,mononuclear cell
2,CL:0000787,memory B cell,rdfs:subClassOf,CL:0000236,B cell
3,CL:0000787,memory B cell,rdfs:subClassOf,CL:0000145,professional antigen presenting cell
4,CL:0000788,naive B cell,rdfs:subClassOf,CL:0000145,professional antigen presenting cell
5,CL:0000788,naive B cell,rdfs:subClassOf,CL:0000236,B cell
6,CL:0000788,naive B cell,rdfs:subClassOf,CL:0000842,mononuclear cell
7,CL:0000798,gamma-delta T cell,rdfs:subClassOf,CL:0000084,T cell
8,CL:0000798,gamma-delta T cell,rdfs:subClassOf,CL:0000842,mononuclear cell
9,CL:0000809,"double-positive, alpha-beta thymocyte",rdfs:subClassOf,CL:0002420,immature T cell


#### Full slim enrichment
Returns a DataFrame that is enriched with inferred relationships between terms in the seed list and in an extended seed list. The extended seed list consists of terms from the seed list and terms from given slim lists, classes tagged with some specified ‘subset’ axiom, with inferred terms via transitive subClassOf queries.

In [10]:
ade.full_slim_enrichment(["blood_and_immune_upper_slim"])

,s,s_label,p,o,o_label
0,CL:0000084,T cell,rdfs:subClassOf,CL:0000842,mononuclear cell
1,CL:0000084,T cell,rdfs:subClassOf,CL:0000542,lymphocyte
2,CL:0000787,memory B cell,rdfs:subClassOf,CL:0000785,mature B cell
3,CL:0000787,memory B cell,rdfs:subClassOf,CL:0000145,professional antigen presenting cell
4,CL:0000787,memory B cell,rdfs:subClassOf,CL:0001201,"B cell, CD19-positive"
...,...,...,...,...,...
159,CL:0002489,double negative thymocyte,rdfs:subClassOf,CL:0000842,mononuclear cell
160,CL:0002489,double negative thymocyte,rdfs:subClassOf,CL:0000084,T cell
161,CL:0002489,double negative thymocyte,rdfs:subClassOf,CL:0002420,immature T cell
162,CL:0002489,double negative thymocyte,rdfs:subClassOf,CL:0000542,lymphocyte


#### Contextual enrichment
Returns a DataFrame that is enriched with inferred relationships between terms in the seed list and in an extended seed list. The extended seed list consists of terms from the seed list and all terms satisfied by some set of existential restrictions in the ubergraph (e.g. part_of some 'kidney').

In [11]:
ade.contextual_slim_enrichment()

,s,s_label,p,o,o_label
0,CL:0000798,gamma-delta T cell,rdfs:subClassOf,CL:0000084,T cell
1,CL:0000809,"double-positive, alpha-beta thymocyte",rdfs:subClassOf,CL:0000084,T cell
2,CL:0000813,memory T cell,rdfs:subClassOf,CL:0000084,T cell
3,CL:0000815,regulatory T cell,rdfs:subClassOf,CL:0000084,T cell
4,CL:0000895,"naive thymus-derived CD4-positive, alpha-beta ...",rdfs:subClassOf,CL:0000084,T cell
5,CL:0000897,"CD4-positive, alpha-beta memory T cell",rdfs:subClassOf,CL:0000084,T cell
6,CL:0000897,"CD4-positive, alpha-beta memory T cell",rdfs:subClassOf,CL:0000813,memory T cell
7,CL:0000900,"naive thymus-derived CD8-positive, alpha-beta ...",rdfs:subClassOf,CL:0000084,T cell
8,CL:0000909,"CD8-positive, alpha-beta memory T cell",rdfs:subClassOf,CL:0000084,T cell
9,CL:0000909,"CD8-positive, alpha-beta memory T cell",rdfs:subClassOf,CL:0000813,memory T cell


### Secondary Example

In [12]:
#Using Integrated Single-nucleus and Single-cell RNA-seq of the Adult Human Kidney dataset
ade = AnndataEnricher.from_file_path("test/data/human_kidney.h5ad")
print(f"""Contexts of the dataset from tissue field are 
{ade._AnndataEnricher__context_list}""")
ade.contextual_slim_enrichment()

Contexts of the dataset from tissue field are 
['UBERON:0000362', 'UBERON:0001225', 'UBERON:0001228', 'UBERON:0002113']


,s,s_label,p,o,o_label
0,CL:0000653,podocyte,rdfs:subClassOf,CL:0002584,renal cortical epithelial cell
1,CL:0000653,podocyte,rdfs:subClassOf,CL:0002681,kidney cortical cell
2,CL:0000653,podocyte,rdfs:subClassOf,CL:1000449,epithelial cell of nephron
3,CL:0000653,podocyte,rdfs:subClassOf,CL:1000450,epithelial cell of glomerular capsule
4,CL:0000653,podocyte,rdfs:subClassOf,CL:1000510,kidney glomerular epithelial cell
...,...,...,...,...,...
132,CL:1001432,kidney collecting duct intercalated cell,rdfs:subClassOf,CL:1000497,kidney cell
133,CL:1001432,kidney collecting duct intercalated cell,rdfs:subClassOf,CL:0005010,renal intercalated cell
134,CL:1001432,kidney collecting duct intercalated cell,rdfs:subClassOf,CL:0002518,kidney epithelial cell
135,CL:1001432,kidney collecting duct intercalated cell,rdfs:subClassOf,CL:1001225,kidney collecting duct cell


## AnndataAnalyzer walkthrough

### Initialization
Let's import the necessary modules and initialize our AnndataAnalyzer

In [13]:
from pandasaurus_cxg.anndata_analyzer import AnndataAnalyzer

In [14]:
# temporarily using a placeholder schema for free text cell types 
ada = AnndataAnalyzer("test/data/test_covid.h5ad", "pandasaurus_cxg/schema/schema.json")

### Analyzer

#### Co-annotation report
Generates a co-annotation report based on the provided schema.

In [15]:
ada.co_annotation_report()

,field_name1,value1,predicate,field_name2,value2
0,author_cell_type,naive B cell,cluster_matches,cell_type,naive B cell
1,author_cell_type,memory B cell,cluster_matches,cell_type,memory B cell
2,author_cell_type,gamma-delta T cell,cluster_matches,cell_type,gamma-delta T cell
3,author_cell_type,plasmablast,cluster_matches,cell_type,plasmablast
4,author_cell_type,regulatory T cell,cluster_matches,cell_type,regulatory T cell
5,author_cell_type,"CD4-positive, alpha-beta memory T cell",cluster_matches,cell_type,"CD4-positive, alpha-beta memory T cell"
6,author_cell_type,"CD8-positive, alpha-beta memory T cell",cluster_matches,cell_type,"CD8-positive, alpha-beta memory T cell"
7,author_cell_type,naive CD8+ T cell,cluster_matches,cell_type,"naive thymus-derived CD8-positive, alpha-beta ..."
8,author_cell_type,naive CD4+ T cell,cluster_matches,cell_type,"naive thymus-derived CD4-positive, alpha-beta ..."
9,author_cell_type,mucosal invariant T cell (MAIT),cluster_matches,cell_type,mucosal invariant T cell


### Secondary Example

In [16]:
ada = AnndataAnalyzer("test/data/human_kidney.h5ad", "pandasaurus_cxg/schema/schema.json")
ada.co_annotation_report()

,field_name1,value1,predicate,field_name2,value2
0,subclass.l3,dPT,cluster_matches,subclass.full,Degenerative Proximal Tubule Epithelial Cell
1,subclass.l3,aPT,cluster_matches,subclass.full,Adaptive / Maladaptive / Repairing Proximal Tu...
2,subclass.l3,M-FIB,cluster_matches,subclass.full,Medullary Fibroblast
3,subclass.l3,MD,cluster_matches,subclass.full,Macula Densa Cell
4,subclass.l3,NKC/T,cluster_matches,subclass.full,Natural Killer Cell / Natural Killer T Cell
...,...,...,...,...,...
415,cell_type,kidney distal convoluted tubule epithelial cell,subcluster_of,class,epithelial cells
416,cell_type,renal interstitial pericyte,subcluster_of,class,stroma cells
417,cell_type,neural cell,cluster_matches,class,neural cells
418,cell_type,parietal epithelial cell,subcluster_of,class,epithelial cells
